In [71]:
import pandas as pd
import numpy as np
from utils import *



In [72]:
datario_tx_precip_alerta_rio = pd.read_csv("database_csv/taxa_precipitacao_alertario.csv")
cor_est_alerta_rio = pd.read_csv("database_csv/estacoes_alertario.csv")

In [73]:
# Criando uma coluna com o horário preciso da medição
datario_tx_precip_alerta_rio["data_hora"] = datario_tx_precip_alerta_rio["data_particao"] + ' ' + datario_tx_precip_alerta_rio["horario"]

# Converter para datetime
datario_tx_precip_alerta_rio['data_hora'] = pd.to_datetime(datario_tx_precip_alerta_rio['data_hora'])
datario_tx_precip_alerta_rio['data_particao'] = pd.to_datetime(datario_tx_precip_alerta_rio['data_particao'])


datario_tx_precip_alerta_rio = datario_tx_precip_alerta_rio.sort_values(by='data_hora', ascending=True)

In [74]:
# Retirando os dias os quais aparecem o valor NAN
datario_tx_precip_alerta_rio = datario_tx_precip_alerta_rio[~pd.isna(datario_tx_precip_alerta_rio["data_hora"])]

In [75]:
# Preencher valores NaN com 0 nas colunas especificadas
colunas_chuva = ['acumulado_chuva_15_min', 'acumulado_chuva_1_h', 'acumulado_chuva_4_h', 'acumulado_chuva_24_h']

datario_tx_precip_alerta_rio[colunas_chuva] = datario_tx_precip_alerta_rio[colunas_chuva].fillna(0)
datario_tx_precip_alerta_rio[colunas_chuva] = datario_tx_precip_alerta_rio[colunas_chuva].astype(float)

In [76]:
datario_tx_precip_alerta_rio_red = datario_tx_precip_alerta_rio.loc[datario_tx_precip_alerta_rio['data_particao'].dt.year >= 2016].copy()

# Supondo que 'datario_tx_precip_alerta_rio_red' seja o nome do seu DataFrame
# Substitua esse nome pelo nome real do seu DataFrame

# Calculando a próxima data usando shift()
datario_tx_precip_alerta_rio_red['proximo_data'] = datario_tx_precip_alerta_rio_red['data_particao'].shift(-1)

datario_tx_precip_alerta_rio_red = datario_tx_precip_alerta_rio_red.dropna(subset=['data_particao', 'proximo_data'])

# Calculando a diferença entre datas consecutivas em minutos
datario_tx_precip_alerta_rio_red['diferenca_minutos'] = (datario_tx_precip_alerta_rio_red['proximo_data'] - datario_tx_precip_alerta_rio_red['data_particao']).dt.total_seconds() / 60

# Convertendo para inteiros
datario_tx_precip_alerta_rio_red['diferenca_minutos'] = datario_tx_precip_alerta_rio_red['diferenca_minutos'].astype(int)

# Retirando linhas onde a diferença é 5 ou 10 minutos
datario_tx_precip_alerta_rio_red = datario_tx_precip_alerta_rio_red[(datario_tx_precip_alerta_rio_red['diferenca_minutos'] != 5) & (datario_tx_precip_alerta_rio_red['diferenca_minutos'] != 10)]

# Se desejar, você pode remover a coluna de diferença de minutos após filtrar as linhas
del datario_tx_precip_alerta_rio_red['diferenca_minutos']
del datario_tx_precip_alerta_rio_red['proximo_data']

In [77]:
datario_tx_precip_alerta_rio_red = processamento_acumulado_chuva(datario_tx_precip_alerta_rio_red)

datario_tx_precip_alerta_rio_red = ordenar_colunas(datario_tx_precip_alerta_rio_red)

datario_tx_precip_alerta_rio_red = converte_mm_por_h(datario_tx_precip_alerta_rio_red)



In [78]:
eventos_chuvosos, dados_eventos, df_result = encontra_chuvas_mais_fortes(datario_tx_precip_alerta_rio_red, 'acumulado_chuva_1_h', quantidade_eventos=10, df_latitudes=cor_est_alerta_rio)

In [79]:
df_result= df_result.rename(columns={
    0: "data_inicio_chuva",
    1: "data_fim_chuva",
    2: "maior_chuva_evento",
    3: "id_estacao_especifica",
    4: "size_point",
    5: "latitude",
    6: "longitude",
    7: "duracao_minutos",
    8: "duracao_horas"
})

In [80]:
df_result

,data_inicio_chuva,data_fim_chuva,maior_chuva_evento,id_estacao_especifica,size_point,latitude,longitude,duracao_minutos,duracao_horas
8_evento_827,2024-02-17 10:55:00,2024-02-20 22:30:00,1049.2,8,100,-22.81806,-43.21028,695.0,11.583333
20_evento_654,2022-03-31 20:00:00,2022-04-03 06:30:00,125.6,20,91,-23.05028,-43.59472,630.0,10.500000
19_evento_222,2018-02-14 19:45:00,2018-02-15 09:45:00,123.2,19,82,-22.97721,-43.39155,840.0,14.000000
18_evento_205,2018-02-14 23:15:00,2018-02-15 15:00:00,109.6,18,73,-22.94556,-43.36278,945.0,15.750000
13_evento_216,2018-02-14 23:15:00,2018-02-15 13:15:00,106.6,13,64,-22.89182,-43.31005,840.0,14.000000
1_evento_26,2016-03-12 19:00:00,2016-03-13 10:00:00,106.4,1,55,-22.99250,-43.23306,900.0,15.000000
3_evento_29,2016-03-12 19:00:00,2016-03-13 10:30:00,105.2,3,46,-22.98583,-43.24500,930.0,15.500000
21_evento_220,2018-02-14 23:15:00,2018-02-15 13:00:00,99.8,21,38,-22.92556,-43.31583,825.0,13.750000
10_evento_208,2018-02-14 23:00:00,2018-02-15 12:45:00,94.8,10,28,-22.87333,-43.33889,825.0,13.750000
17_evento_337,2019-04-08 17:30:00,2019-04-09 22:45:00,92.2,17,19,-23.00849,-43.29965,315.0,5.250000


In [81]:
# Criar um dicionário que mapeia os IDs das estações para os nomes das estações
mapa_estacoes = dict(zip(cor_est_alerta_rio['id_estacao'], cor_est_alerta_rio['estacao']))

# Função para obter o nome da estação com base no ID da estação
def obter_nome_estacao(id_estacao):
    return mapa_estacoes.get(id_estacao, "Estação Desconhecida")  # Retorna o nome da estação se existir no dicionário, caso contrário retorna "Estação Desconhecida"

# Adicionar a nova coluna 'nome' ao DataFrame df_result
df_result['nome'] = df_result['id_estacao_especifica'].map(obter_nome_estacao)

In [82]:
import plotly.express as px

fig = px.bar(df_result, x=df_result.index, y='maior_chuva_evento', title="Maiores chuvas")
fig.update_layout(xaxis_title="Evento em questão", yaxis_title="maior quantidade em mm/h")

fig.show()


In [84]:
# Plotar no mapa
fig = px.scatter_mapbox(df_result, lat="latitude", lon="longitude", hover_data=["nome", "data_inicio_chuva", "data_fim_chuva"],
                        color_discrete_sequence=["red"], zoom=10, size="size_point", height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()